# Przydatne linki
### bit.ly/symulator-kod

### bit.ly/symulator-model

In [1]:
from flask import Flask
from io import BytesIO
import eventlet.wsgi
import eventlet
import socketio
import base64
from PIL import Image
import numpy as np

import keras

from keras.models import load_model

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
"""korzystamy z wyuczonego modelu"""
model = load_model('input/my_model_2.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalize (Lambda)           (None, 60, 80, 3)         0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 58, 78, 64)        1792      
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 56, 76, 32)        18464     
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 28, 38, 32)        0         
_________________________________________________________________
conv2d_69 (Conv2D)           (None, 26, 36, 16)        4624      
_______________________________________

In [3]:
"""serwer web do odbierania zapytan z symulatora"""
sio = socketio.Server()
app = Flask(__name__)

In [4]:
"""funkcja wysylajaca rozkazy do symulatora"""
def send_control(steering_angle, throttle):
    sio.emit("steer", data={'steering_angle': str(steering_angle),
                            'throttle': str(throttle) }, skip_sid=True)

In [5]:
""" funkcja pomocnicza - zmienia rozmiar obrazu wejsciowiego na 60 x 80 x 3"""
def process_image(img):
    return img[10:130:2 , ::4, : ]

@sio.on('telemetry')
def telemetry(sid, data):
    if data:
        speed = float(data["speed"])
        image_str = data["image"]#obraz jak string
        
        #print(data.keys())# 4 wymiary aktualny skręt, polozenie pedalu gazu, prędkosc, obraz - dane z symulatora
        #print(type(image_str))#obraz jako string
        
        #decodowanie obrazu
        decoded = base64.b64decode(image_str)
        
        #konwertujemy do obrazu
        image = Image.open(BytesIO(decoded))
        
        #obraz konwertujemy do macierzy
        imgae_array = np.asarray(image)
        #print(imgae_array.shape)#obraz o wymiarach 160 wysokosc x 320 szerokosc x 3 kanały (RGB)
        
        #model input 60x80x3
        #plt.imshow(imgae_array)
        #plt.show()
        
        #zmniejszamy obraz 
        img = process_image(imgae_array)
        img_batch = np.expand_dims(img,axis = 0)#dodanie 4 wymiaru , 1 zdjecie
        
        #predykcja modelu i zmiana na float - skret kierownicy
        steering_angle = float(model.predict(img_batch))
        
        #print(steering_angle) 
        #steering_angle = 0.0 #kat skretu watosci -1..1 (-1 -lewo, 1 -prawo)
        throttle = 0.1 #wcisniecie gazu wartosci 0..1 wartosci,  -1..0 oznacza hamowanie
        
        #print(throttle,steering_angle)
        
        if speed < 10:
            throttle = 0.6
        if speed > 17:
            throttle = -0.1
        send_control(steering_angle, throttle)
    else:
        sio.emit('manual', data={}, skip_sid=True)

##uruchamia serwer i blokuje komórke
app = socketio.Middleware(sio, app)
eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

(163656) wsgi starting up on http://0.0.0.0:4567
(163656) accepted ('127.0.0.1', 59282)
127.0.0.1 - - [17/May/2019 15:11:36] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 62.270511
(163656) accepted ('127.0.0.1', 59303)
127.0.0.1 - - [17/May/2019 15:13:41] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 91.124766
(163656) accepted ('127.0.0.1', 59325)
127.0.0.1 - - [17/May/2019 15:14:59] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 72.746071
wsgi exiting
(163656) wsgi exited, is_accepting=True
